<a href="https://colab.research.google.com/github/MasoudMoeini/Image-blur-detection/blob/main/image_blur_detection_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib . pyplot as plt
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [16]:
!unzip -qq train.zip

replace __MACOSX/train/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [17]:
!ls train

images	labels


In [ ]:
from keras.preprocessing.image import array_to_img
imgb_from_array=array_to_img(imgb_to_array)
plt.imshow(imgb_from_array)

In [7]:
def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 224, 224, 3))
    return array

In [ ]:
train_data = preprocess(train_data)

In [9]:
input = layers.Input(shape=(224, 224, 3))
# Encoder

# Conv1
# Input Tensor Shape: [batch_size, 224, 224, 3]
# Output Tensor Shape: [batch_size, 224, 224, 32]
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)

# conv2
# Input Tensor Shape: [batch_size, 224, 224, 32]
# Output Tensor Shape: [batch_size, 224, 224, 32]
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)

# pool1
# Input Tensor Shape: [batch_size, 224, 224, 32]
# Output Tensor Shape: [batch_size, 112, 112, 32]

x = layers.MaxPooling2D((2, 2), padding="same")(x)

#conv3
# Input Tensor Shape: [batch_size, 112, 112, 32]
# Output Tensor Shape: [batch_size, 112, 112, 64]
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

#conv4
# Input Tensor Shape: [batch_size, 112, 112, 64]
# Output Tensor Shape: [batch_size, 112, 112, 64]
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

#pool2
# Input Tensor Shape: [batch_size, 112, 112, 64]
# Output Tensor Shape: [batch_size, 56, 56, 64]
x = layers.MaxPooling2D((2, 2), padding="same")(x)

#conv5
# Input Tensor Shape: [batch_size, 56, 56, 64]
# Output Tensor Shape: [batch_size, 56, 56, 128]
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

#conv6
# Input Tensor Shape: [batch_size, 56, 56, 128]
# Output Tensor Shape: [batch_size, 56, 56, 128]
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

#pool3
# Input Tensor Shape: [batch_size, 56, 56, 128]
# Output Tensor Shape: [batch_size, 28, 28, 128]
x = layers.MaxPooling2D((2, 2), padding="same")(x)

#conv7
# Input Tensor Shape: [batch_size, 28, 28, 128]
# Output Tensor Shape: [batch_size, 28, 28, 256]
x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)

#conv8
# Input Tensor Shape: [batch_size, 28, 28, 256]
# Output Tensor Shape: [batch_size, 28, 28, 256]
x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)

#pool4
# Input Tensor Shape: [batch_size, 28, 28, 256]
# Output Tensor Shape: [batch_size, 14, 14, 256]
x = layers.MaxPooling2D((2, 2), padding="same")(x)
#conv9
# Input Tensor Shape: [batch_size, 14, 14, 256]
# Output Tensor Shape: [batch_size, 14, 14, 512]
x = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(x)
#pool5
# Input Tensor Shape: [batch_size, 14, 14, 512]
# Output Tensor Shape: [batch_size, 7, 7, 512]
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = layers.Conv2DTranspose(512, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(256, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=1, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(16, (3, 3), strides=1, activation="sigmoid", padding="same")(x)
x = layers.Conv2DTranspose(3, (3, 3), strides=1, activation="sigmoid", padding="same")(x)

In [10]:
encoder_decoder = Model(input, x)

In [11]:
encoder_decoder.compile(keras.optimizers.Adam(learning_rate=1e-3),
                        loss=tf.keras.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()],)
encoder_decoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      36928     
                                                             

In [ ]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

encoder_decoder.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

In [ ]:
encoder_decoder.fit(
    x=train_data,
    y=train_data,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
)
from keras.models import load_model
encoder_decoder.save('encoderdecoder_model.h5')